In [1]:
# Import des bibliothèques requises
import numpy as np
import vrplib
import yaml
import random
import math
import time
import os
from collections import deque, defaultdict
from copy import deepcopy
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès !")


Bibliothèques importées avec succès !


In [2]:
# Chargement de la configuration (si disponible)
config_path = 'config.yaml' if os.path.exists('config.yaml') else 'config_hga.yaml'
try:
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print("Configuration chargée depuis:", config_path)
except Exception as e:
    print("Aucune configuration trouvée; utilisation de paramètres par défaut.", e)
    config = {
        'general': {'random_seed': 42, 'time_limit_seconds': 30, 'verbose': True},
        'alns': {'weights_initial': 1.0, 'reaction_factor': 0.1},
        'simulated_annealing': {'initial_temperature': 1000.0, 'final_temperature': 0.1, 'alpha': 0.95, 'iterations_per_temperature': 50},
        'local_search': {'max_iterations': 2000, 'max_iterations_without_improvement': 500},
        'vnd': {'neighborhoods': ['relocate', 'swap', 'two_opt', 'cross_exchange'], 'max_iterations_without_improvement': 50},
        'tabu_search': {'tabu_tenure': 10, 'tabu_tenure_random_range': 2, 'aspiration_enabled': True},
        'quality': {'target_gap_percentage': 7.0}
    }

print("Config :")
try:
    print(yaml.dump(config, default_flow_style=False))
except:
    print(config)

Configuration chargée depuis: config.yaml
Config :
general:
  random_seed: 42
  time_limit_seconds: 300
  verbose: true
initial_solution:
  method: nearest_neighbor
  randomness: 0.1
local_search:
  max_iterations: 1000
  max_iterations_without_improvement: 200
quality:
  penalty_capacity_violation: 10000
  target_gap_percentage: 7.0
simulated_annealing:
  alpha: 0.95
  final_temperature: 0.1
  initial_temperature: 1000.0
  iterations_per_temperature: 100
tabu_search:
  aspiration_enabled: true
  tabu_tenure: 20
  tabu_tenure_random_range: 10
vnd:
  max_iterations_without_improvement: 50
  neighborhoods:
  - swap
  - relocate
  - two_opt
  - cross_exchange



In [3]:
# Fixer la graine aléatoire pour la reproductibilité
seed = config['general'].get('random_seed', 42)
random.seed(seed)
np.random.seed(seed)
print(f"Graine aléatoire fixée : {seed}")

Graine aléatoire fixée : 42


## Structures de données et fonctions auxiliaires (avec fenêtres de temps)

Nous introduisons une représentation de solution qui tient compte des fenêtres de temps et des temps de service.

In [4]:
class CVRPTWSolution:
    """Représentation d'une solution CVRPTW avec routes, coût et faisabilité temps/capacité."""

    def __init__(self, routes: List[List[int]], distance_matrix: np.ndarray, travel_time_matrix: Optional[np.ndarray] = None):
        self.routes = routes
        self.distance_matrix = distance_matrix
        self.travel_time_matrix = travel_time_matrix if travel_time_matrix is not None else distance_matrix
        self.cost = self.calculate_cost()

    def calculate_cost(self) -> float:
        """Coût total = distance totale parcourue."""
        total_cost = 0
        for route in self.routes:
            if len(route) == 0:
                continue
            total_cost += self.distance_matrix[0, route[0]]
            for i in range(len(route) - 1):
                total_cost += self.distance_matrix[route[i], route[i+1]]
            total_cost += self.distance_matrix[route[-1], 0]
        return total_cost

    def update_cost(self):
        """Recalculer le coût (distance)."""
        self.cost = self.calculate_cost()

    def is_feasible(self, instance: Dict, capacity: int) -> bool:
        """Vérifier la faisabilité: capacité et fenêtres de temps."""
        demands = np.array(instance['demand'])
        tw = np.array(instance.get('time_window'))
        earliest = tw[:, 0]
        latest = tw[:, 1]
        dimension = len(demands)
        service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
        
        for route in self.routes:
            if len(route) == 0:
                continue
            # Vérifier capacité
            if sum(demands[c] for c in route) > capacity:
                return False
            # Vérifier fenêtres de temps et service times
            time = 0
            time += self.travel_time_matrix[0, route[0]]
            arrival = time
            start = max(arrival, earliest[route[0]])
            if start > latest[route[0]]:
                return False
            time = start + service_times[route[0]]
            for i in range(len(route) - 1):
                a = route[i]
                b = route[i + 1]
                time += self.travel_time_matrix[a, b]
                arrival = time
                start = max(arrival, earliest[b])
                if start > latest[b]:
                    return False
                time = start + service_times[b]
        return True

    def copy(self):
        return CVRPTWSolution([route.copy() for route in self.routes], self.distance_matrix, self.travel_time_matrix)

print("Classe CVRPTWSolution définie !")

Classe CVRPTWSolution définie !


In [5]:
def calculate_distance_and_time_matrices(instance: Dict) -> Tuple[np.ndarray, np.ndarray]:
    """Renvoie matrices distances et temps (travail simple: vitesse = 1)."""
    if 'edge_weight' in instance:
        dist = instance['edge_weight']
        # If given as list-of-lists, convert to numpy
        dist = np.array(dist)
    else:
        coords = np.array(instance['node_coord'])
        n = len(coords)
        dist = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    dist[i, j] = math.hypot(*(coords[i] - coords[j]))
    # For these instances, we assume travel time == distance (vitesse unitaire)
    travel_time = dist.copy()
    return dist, travel_time

print("Fonction de calcul des matrices de distance et temps définie !")

Fonction de calcul des matrices de distance et temps définie !


## Construction d'une solution initiale (Nearest Neighbor respectant TW)

La heuristique choisit un client faisable (capacité & fenêtre de temps) le plus proche du noeud courant.

In [6]:
def nearest_neighbor_tw_solution(instance: Dict, distance_matrix: np.ndarray, travel_time_matrix: np.ndarray) -> CVRPTWSolution:
    demands = np.array(instance['demand'])
    capacity = instance['capacity']

    # FIXED: safe extraction of time windows (no boolean eval of arrays)
    tw = None
    for key in ['time_window', 'time_windows', 'tw']:
        val = instance.get(key)
        if val is not None:
            tw = val
            break

    if tw is None:
        raise KeyError("Time windows not found in instance")

    tw = np.array(tw)

    earliest = tw[:, 0]
    latest = tw[:, 1]

    dimension = len(demands)

    service_times = np.array(
        instance.get('service_time',
        instance.get('service_times', [0] * dimension))
    )

    unvisited = set(range(1, dimension))
    routes = []

    while unvisited:
        route = []
        current_load = 0
        current_node = 0
        current_time = 0

        depot_earliest = earliest[0] if len(earliest) > 0 else 0
        if current_time < depot_earliest:
            current_time = depot_earliest

        while True:
            best_customer = None
            best_metric = float('inf')

            for c in list(unvisited):
                if current_load + demands[c] > capacity:
                    continue

                travel = travel_time_matrix[current_node, c]
                arrival = current_time + travel
                start = max(arrival, earliest[c])

                if start > latest[c]:
                    continue

                metric = distance_matrix[current_node, c] + (start - earliest[c]) * 0.1

                if metric < best_metric:
                    best_metric = metric
                    best_customer = c

            if best_customer is None:
                break

            route.append(best_customer)
            unvisited.remove(best_customer)

            current_load += demands[best_customer]
            travel = travel_time_matrix[current_node, best_customer]
            arrival = current_time + travel
            current_time = max(arrival, earliest[best_customer]) + service_times[best_customer]
            current_node = best_customer

        if route:
            routes.append(route)

    return CVRPTWSolution(routes, distance_matrix, travel_time_matrix)
print("Fonction de solution du plus proche voisin avec fenêtres de temps définie !")

Fonction de solution du plus proche voisin avec fenêtres de temps définie !


## Opérateurs de voisinage (respectant les fenêtres de temps)

Opérateurs: swap, relocate, 2-opt (intra-route), cross-exchange. Chaque opérateur teste la faisabilité temporelle des routes modifiées.

In [7]:
def time_feasible_for_route(route: List[int], travel_time_matrix: np.ndarray, tw_earliest: np.ndarray, tw_latest: np.ndarray, service_times: np.ndarray, depot_start_time: float = 0.0) -> bool:
    # Simule la route et vérifie les fenêtres de temps
    time = depot_start_time
    if len(route) == 0:
        return True
    time += travel_time_matrix[0, route[0]]
    arrival = time
    start = max(arrival, tw_earliest[route[0]])
    if start > tw_latest[route[0]]: return False
    time = start + service_times[route[0]]
    for i in range(len(route) - 1):
        a = route[i]
        b = route[i + 1]
        time += travel_time_matrix[a, b]
        arrival = time
        start = max(arrival, tw_earliest[b])
        if start > tw_latest[b]: return False
        time = start + service_times[b]
    return True

def swap_operator_tw(solution: CVRPTWSolution, instance: Dict) -> Optional[CVRPTWSolution]:
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    tw = np.array(instance.get('time_window'))
    earliest = tw[:, 0]
    latest = tw[:, 1]
    dimension = len(demands)
    service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
    if len(solution.routes) < 2: return None
    best_solution = None
    best_cost = solution.cost
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            if len(route_i) == 0 or len(route_j) == 0: continue
            for pos_i in range(len(route_i)):
                for pos_j in range(len(route_j)):
                    new_solution = solution.copy()
                    a = new_solution.routes[i][pos_i]
                    b = new_solution.routes[j][pos_j]
                    new_solution.routes[i][pos_i], new_solution.routes[j][pos_j] = b, a
                    # capacity feasibility
                    if sum(demands[c] for c in new_solution.routes[i]) > capacity: continue
                    if sum(demands[c] for c in new_solution.routes[j]) > capacity: continue
                    # time windows feasibility for both routes
                    if not time_feasible_for_route(new_solution.routes[i], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                    if not time_feasible_for_route(new_solution.routes[j], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                    new_solution.update_cost()
                    if new_solution.cost < best_cost:
                        best_cost = new_solution.cost
                        best_solution = new_solution
    return best_solution

def relocate_operator_tw(solution: CVRPTWSolution, instance: Dict) -> Optional[CVRPTWSolution]:
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    tw = np.array(instance.get('time_window'))
    earliest = tw[:, 0]
    latest = tw[:, 1]
    dimension = len(demands)
    service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
    best_solution = None
    best_cost = solution.cost
    for i in range(len(solution.routes)):
        route_i = solution.routes[i]
        for pos_i in range(len(route_i)):
            customer = route_i[pos_i]
            for j in range(len(solution.routes)):
                if i == j: continue
                for pos_j in range(len(solution.routes[j]) + 1):
                    new_solution = solution.copy()
                    removed = new_solution.routes[i].pop(pos_i)
                    new_solution.routes[j].insert(pos_j, removed)
                    if sum(demands[c] for c in new_solution.routes[j]) > capacity: continue
                    if not time_feasible_for_route(new_solution.routes[i], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                    if not time_feasible_for_route(new_solution.routes[j], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                    new_solution.update_cost()
                    if new_solution.cost < best_cost:
                        best_cost = new_solution.cost
                        best_solution = new_solution
    return best_solution

def two_opt_operator_tw(solution: CVRPTWSolution, instance: Dict) -> Optional[CVRPTWSolution]:
    tw = np.array(instance.get('time_window'))
    earliest = tw[:, 0]
    latest = tw[:, 1]
    dimension = len(np.array(instance['demand']))
    service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
    best_solution = None
    best_cost = solution.cost
    for route_idx in range(len(solution.routes)):
        route = solution.routes[route_idx]
        if len(route) < 2: continue
        for i in range(len(route) - 1):
            for j in range(i + 1, len(route)):
                new_solution = solution.copy()
                new_solution.routes[route_idx][i:j+1] = list(reversed(new_solution.routes[route_idx][i:j+1]))
                if not time_feasible_for_route(new_solution.routes[route_idx], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                new_solution.update_cost()
                if new_solution.cost < best_cost:
                    best_cost = new_solution.cost
                    best_solution = new_solution
    return best_solution

def cross_exchange_operator_tw(solution: CVRPTWSolution, instance: Dict) -> Optional[CVRPTWSolution]:
    tw = np.array(instance.get('time_window'))
    earliest = tw[:, 0]
    latest = tw[:, 1]
    dimension = len(np.array(instance['demand']))
    service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
    if len(solution.routes) < 2: return None
    best_solution = None
    best_cost = solution.cost
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            if len(route_i) < 2 or len(route_j) < 2: continue
            for seg_len in [1, 2]:
                for pos_i in range(len(route_i) - seg_len + 1):
                    for pos_j in range(len(route_j) - seg_len + 1):
                        new_solution = solution.copy()
                        seg_i = new_solution.routes[i][pos_i:pos_i+seg_len]
                        seg_j = new_solution.routes[j][pos_j:pos_j+seg_len]
                        new_solution.routes[i][pos_i:pos_i+seg_len] = seg_j
                        new_solution.routes[j][pos_j:pos_j+seg_len] = seg_i
                        if not time_feasible_for_route(new_solution.routes[i], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                        if not time_feasible_for_route(new_solution.routes[j], new_solution.travel_time_matrix, earliest, latest, service_times): continue
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    return best_solution

print("Opérateurs de voisinage CVRPTW définis !")

Opérateurs de voisinage CVRPTW définis !


## Variable Neighborhood Descent (VND) adapté au CVRPTW (TW)

Le VND parcourt un ordre de voisinages et applique des opérateurs locaux jusqu'à ne plus s'améliorer.

In [8]:
def vnd_tw(solution: CVRPTWSolution, instance: Dict) -> CVRPTWSolution:
    operators = {
        'swap': lambda s: swap_operator_tw(s, instance),
        'relocate': lambda s: relocate_operator_tw(s, instance),
        'two_opt': lambda s: two_opt_operator_tw(s, instance),
        'cross_exchange': lambda s: cross_exchange_operator_tw(s, instance)
    }
    neighborhood_order = config['vnd']['neighborhoods'] if 'vnd' in config else list(operators.keys())
    max_no_improve = config['vnd'].get('max_iterations_without_improvement', 50)
    current_solution = solution
    k = 0
    no_improve = 0
    while k < len(neighborhood_order) and no_improve < max_no_improve:
        op_name = neighborhood_order[k]
        op = operators[op_name]
        new_solution = op(current_solution)
        if new_solution is not None and new_solution.cost < current_solution.cost:
            current_solution = new_solution
            k = 0
            no_improve = 0
        else:
            k += 1
            no_improve += 1
    return current_solution

print("VND pour CVRPTW défini !")

VND pour CVRPTW défini !


## ALNS (Adaptive Large Neighborhood Search)

Nous implémentons un ALNS simple: ensemble d'opérateurs de destruction et de réparation, pondérations adaptatives et récompenses.

In [9]:
def remove_random(solution: CVRPTWSolution, instance: Dict, num_remove: int) -> Tuple[CVRPTWSolution, List[int]]:
    # Remove random customers across routes
    all_customers = [c for r in solution.routes for c in r]
    if not all_customers:
        return solution.copy(), []
    removed = random.sample(all_customers, min(num_remove, len(all_customers)))
    new_sol = solution.copy()
    for c in removed:
        # remove c from its route
        for r in new_sol.routes:
            if c in r: r.remove(c); break
    return new_sol, removed

def remove_worst(solution: CVRPTWSolution, instance: Dict, num_remove: int) -> Tuple[CVRPTWSolution, List[int]]:
    # Remove customers that contribute most to cost (simple heuristic)
    # compute marginal cost of each customer as distance to predecessor+successor - direct distance between predecessor and successor
    marginal = []
    for r in solution.routes:
        for idx, c in enumerate(r):
            prev = 0 if idx == 0 else r[idx-1]
            nxt = 0 if idx == len(r)-1 else r[idx+1]
            val = solution.distance_matrix[prev, c] + solution.distance_matrix[c, nxt] - solution.distance_matrix[prev, nxt]
            marginal.append((val, c))
    marginal.sort(reverse=True)
    removed = [c for (_, c) in marginal[:num_remove]] if marginal else []
    new_sol = solution.copy()
    for c in removed:
        for r in new_sol.routes:
            if c in r: r.remove(c); break
    return new_sol, removed

def greedy_repair(solution: CVRPTWSolution, instance: Dict, removed: List[int]) -> CVRPTWSolution:
    # Simple greedy insertion by lowest insertion cost respecting capacity & TW
    new_sol = solution.copy()
    demands = np.array(instance['demand'])
    tw = np.array(instance.get('time_window'))
    earliest = tw[:, 0]
    latest = tw[:, 1]
    dimension = len(demands)
    service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
    for c in removed:
        best = None
        best_cost = float('inf')
        # try insert into all routes + new route at end
        for r_idx in range(len(new_sol.routes)):
            for pos in range(len(new_sol.routes[r_idx]) + 1):
                trial = new_sol.copy()
                trial.routes[r_idx].insert(pos, c)
                if sum(demands[x] for x in trial.routes[r_idx]) > instance['capacity']: continue
                if not time_feasible_for_route(trial.routes[r_idx], trial.travel_time_matrix, earliest, latest, service_times): continue
                trial.update_cost()
                if trial.cost < best_cost:
                    best_cost = trial.cost
                    best = trial
        # try to put in new route
        trial = new_sol.copy()
        trial.routes.append([c])
        if sum(demands[x] for x in trial.routes[-1]) <= instance['capacity'] and time_feasible_for_route(trial.routes[-1], trial.travel_time_matrix, earliest, latest, service_times):
            trial.update_cost()
            if trial.cost < best_cost:
                best_cost = trial.cost
                best = trial
        if best is not None:
            new_sol = best
        else:
            # failed to reinsert feasibly; skip this customer (shouldn't happen if instance feasible)
            pass
    return new_sol

def regret2_repair(solution: CVRPTWSolution, instance: Dict, removed: List[int]) -> CVRPTWSolution:
    # Regret-2 insertion: for each customer compute best two insertion costs and choose the one with maximal regret
    new_sol = solution.copy()
    demands = np.array(instance['demand'])
    tw = np.array(instance.get('time_window'))
    earliest = tw[:, 0]
    latest = tw[:, 1]
    dimension = len(demands)
    service_times = np.array(instance.get('service_time', instance.get('service_times', [0] * dimension)))
    to_insert = removed.copy()
    while to_insert:
        best_customer = None
        best_pos = None
        best_increase = float('inf')
        regrets = []
        for c in to_insert:
            insert_options = []
            for r_idx in range(len(new_sol.routes)):
                for pos in range(len(new_sol.routes[r_idx]) + 1):
                    trial = new_sol.copy()
                    trial.routes[r_idx].insert(pos, c)
                    if sum(demands[x] for x in trial.routes[r_idx]) > instance['capacity']: continue
                    if not time_feasible_for_route(trial.routes[r_idx], trial.travel_time_matrix, earliest, latest, service_times): continue
                    trial.update_cost()
                    insert_options.append((trial.cost - new_sol.cost, r_idx, pos, trial))
            # also try new route insertion
            trial = new_sol.copy()
            trial.routes.append([c])
            if sum(demands[x] for x in trial.routes[-1]) <= instance['capacity'] and time_feasible_for_route(trial.routes[-1], trial.travel_time_matrix, earliest, latest, service_times):
                trial.update_cost(); insert_options.append((trial.cost - new_sol.cost, len(trial.routes)-1, 0, trial))
            if not insert_options:
                # can't insert c feasibly with current solution; skip for now
                continue
            insert_options.sort()
            if len(insert_options) == 1:
                regret = insert_options[0][0]
            else:
                regret = insert_options[1][0] - insert_options[0][0]
            regrets.append((regret, insert_options[0], c))
        if not regrets:
            break
        # choose the one with max regret
        regrets.sort(reverse=True)
        _, best_choice, chosen_customer = regrets[0]
        _, r_idx, pos, trial = best_choice
        new_sol = trial
        if chosen_customer in to_insert: to_insert.remove(chosen_customer)
    return new_sol

print("Opérateurs ALNS définis !")

Opérateurs ALNS définis !


## Liste Tabou (identique au CVRP mais stocke mouvements liés à clients)

La liste tabou empêche de réappliquer certains mouvements pour une période donnée, mais autorise l'aspiration si l'amélioration est significative.

In [10]:
class TabuList:
    def __init__(self, tenure: int):
        self.tenure = tenure
        self.tabu_dict = {}
        self.current_iteration = 0
    def add(self, move: Tuple, tenure_variation: int = 0):
        actual_tenure = self.tenure + random.randint(-tenure_variation, tenure_variation)
        self.tabu_dict[move] = self.current_iteration + actual_tenure
    def is_tabu(self, move: Tuple) -> bool:
        if move not in self.tabu_dict: return False
        return self.tabu_dict[move] > self.current_iteration
    def increment_iteration(self):
        self.current_iteration += 1
        expired = [m for m, exp in self.tabu_dict.items() if exp <= self.current_iteration]
        for m in expired: del self.tabu_dict[m]

print("Classe TabuList définie !")

Classe TabuList définie !


## Simulated Annealing couplé à ALNS + Tabu (génère voisins via ALNS et contrôle par Tabu)

La boucle utilise ALNS pour générer solutions voisines puis décide de les accepter selon le critère SA; la liste tabou est utilisée pour interdire certains mouvements fréquents.

In [11]:
def acceptance_probability(current_cost: float, new_cost: float, temperature: float) -> float:
    if new_cost < current_cost:
        return 1.0
    try:
        return math.exp((current_cost - new_cost) / temperature)
    except OverflowError:
        return 0.0


def alns_generate_neighbor(solution: CVRPTWSolution, instance: Dict, alns_state: Dict):
    dimension = len(np.array(instance['demand']))
    min_remove = 1
    max_remove = max(1, int(0.2 * (dimension - 1)))
    num_remove = random.randint(min_remove, max_remove)

    destroy_ops = alns_state['destroy_ops']
    repair_ops = alns_state['repair_ops']
    w_destroy = alns_state['weights_destroy']
    w_repair = alns_state['weights_repair']

    # destroy select
    total = sum(w_destroy.values())
    r = random.random() * total
    upto = 0
    chosen_destroy = None
    for name in destroy_ops:
        upto += w_destroy[name]
        if upto >= r:
            chosen_destroy = name
            break

    # repair select
    total = sum(w_repair.values())
    r = random.random() * total
    upto = 0
    chosen_repair = None
    for name in repair_ops:
        upto += w_repair[name]
        if upto >= r:
            chosen_repair = name
            break

    # destruction
    if chosen_destroy == 'random_remove':
        partial, removed = remove_random(solution, instance, num_remove)
    elif chosen_destroy == 'worst_remove':
        partial, removed = remove_worst(solution, instance, num_remove)
    else:
        partial, removed = remove_random(solution, instance, num_remove)

    # repair
    if chosen_repair == 'greedy':
        neighbor = greedy_repair(partial, instance, removed)
    elif chosen_repair == 'regret2':
        neighbor = regret2_repair(partial, instance, removed)
    else:
        neighbor = greedy_repair(partial, instance, removed)

    return neighbor, {'destroy': chosen_destroy, 'repair': chosen_repair}


def simulated_annealing_with_alns_tabu(initial_solution: CVRPTWSolution, instance: Dict, time_limit: float = None):
    temp = config['simulated_annealing']['initial_temperature']
    final_temp = config['simulated_annealing']['final_temperature']
    alpha = config['simulated_annealing']['alpha']
    iterations_per_temp = config['simulated_annealing']['iterations_per_temperature']

    tabu_tenure = config['tabu_search']['tabu_tenure']
    tabu_tenure_variation = config['tabu_search']['tabu_tenure_random_range']
    aspiration_enabled = config['tabu_search']['aspiration_enabled']

    max_iterations = config['local_search']['max_iterations']
    max_no_improve = config['local_search']['max_iterations_without_improvement']

    alns_state = {
        'destroy_ops': ['random_remove', 'worst_remove'],
        'repair_ops': ['greedy', 'regret2'],
        'weights_destroy': {'random_remove': 1.0, 'worst_remove': 1.0},
        'weights_repair': {'greedy': 1.0, 'regret2': 1.0},
        'scores': defaultdict(int)
    }

    tabu_list = TabuList(tabu_tenure)

    current_solution = initial_solution
    best_solution = current_solution.copy()

    cost_history = [best_solution.cost]
    no_improve = 0
    total_iterations = 0
    start_time = time.time()

    while temp > final_temp and total_iterations < max_iterations:
        if time_limit and (time.time() - start_time) > time_limit:
            break
        if no_improve >= max_no_improve:
            break

        for _ in range(iterations_per_temp):
            if total_iterations % 50 == 0:
                current_solution = vnd_tw(current_solution, instance)

            neighbor, ops = alns_generate_neighbor(current_solution, instance, alns_state)
            if neighbor is None:
                continue

            move = (ops['destroy'], ops['repair'], hash(str(neighbor.routes)))
            is_tabu = tabu_list.is_tabu(move)
            aspiration = aspiration_enabled and neighbor.cost < best_solution.cost

            if (not is_tabu) or aspiration:
                if random.random() < acceptance_probability(current_solution.cost, neighbor.cost, temp):
                    current_solution = neighbor
                    tabu_list.add(move, tabu_tenure_variation)

                    if current_solution.cost < best_solution.cost:
                        best_solution = current_solution.copy()
                        cost_history.append(best_solution.cost)
                        no_improve = 0
                        alns_state['scores'][(ops['destroy'], ops['repair'])] += 1
                    else:
                        no_improve += 1

            tabu_list.increment_iteration()
            total_iterations += 1

        temp *= alpha

        # ALNS weight update
        rf = config.get('alns', {}).get('reaction_factor', 0.1)
        for (d, r), s in list(alns_state['scores'].items()):
            if s > 0:
                alns_state['weights_destroy'][d] = (1 - rf) * alns_state['weights_destroy'][d] + rf * s
                alns_state['weights_repair'][r] = (1 - rf) * alns_state['weights_repair'][r] + rf * s
                alns_state['scores'][(d, r)] = 0

    return best_solution, cost_history

print("Simulated annealing + ALNS + Tabu défini !")

Simulated annealing + ALNS + Tabu défini !


## Fonction principale de résolution (pour une instance Solomon)

Lit l'instance, construit la solution initiale, applique VND et la boucle SA+ALNS+Tabu, et sauvegarde la solution.

In [12]:
def solve_cvrptw(instance_path: str) -> Dict:
    print(f"\n{'='*60}")
    print(f"Solving: {os.path.basename(instance_path)}")
    print(f"{'='*60}")

    try:
        instance = vrplib.read_instance(instance_path, instance_format='solomon')
    except Exception:
        instance = vrplib.read_instance(instance_path)

    dist, travel_time = calculate_distance_and_time_matrices(instance)
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    dimension = len(demands)

    print(f"Dimension: {dimension} nodes; Capacity: {capacity}")

    initial_solution = nearest_neighbor_tw_solution(instance, dist, travel_time)
    print(f"Initial cost: {initial_solution.cost:.2f}; Routes: {len(initial_solution.routes)}")

    improved = vnd_tw(initial_solution, instance)
    print(f"Après VND: {improved.cost:.2f}")

    time_limit = config['general'].get('time_limit_seconds', 30)
    start_time = time.time()

    final_solution, cost_history = simulated_annealing_with_alns_tabu(improved, instance, time_limit)
    elapsed = time.time() - start_time

    print(f"Final cost: {final_solution.cost:.2f}")
    print(f"Final routes: {len(final_solution.routes)}; Time: {elapsed:.2f} sec")

    optimal_cost = None
    gap_pct = None
    sol_path = instance_path.replace('.txt', '.sol').replace('.vrp', '.sol')

    if os.path.exists(sol_path):
        try:
            optimal = vrplib.read_solution(sol_path)
            optimal_cost = optimal.get('cost')
            if optimal_cost:
                gap_pct = ((final_solution.cost - optimal_cost) / optimal_cost) * 100
            print(f"Optimal: {optimal_cost}; Gap: {gap_pct:.2f}%")
        except Exception as e:
            print("Impossible de lire la solution optimale :", e)

    return {
        'instance_name': os.path.basename(instance_path),
        'solution': final_solution,
        'cost': final_solution.cost,
        'optimal_cost': optimal_cost,
        'gap_pct': gap_pct,
        'n_routes': len(final_solution.routes),
        'time_seconds': elapsed,
        'cost_history': cost_history
    }

print("Fonction de résolution CVRPTW définie !")

Fonction de résolution CVRPTW définie !


## Sauvegarde de la solution (format VRPLIB simplifié)

Cette cellule écrit une solution en fichier texte (routes + coût).

In [13]:
def save_solution_tw(result: Dict, output_dir: str = 'solutions_TW') -> str:
    os.makedirs(output_dir, exist_ok=True)
    name = result['instance_name'].replace('.txt', '').replace('.vrp', '')
    out_path = os.path.join(output_dir, f"{name}_computed.sol")
    routes = result['solution'].routes
    with open(out_path, 'w') as f:
        for i, r in enumerate(routes, 1):
            f.write(f"Route #{i}: {' '.join(map(str, r))}\n")
        f.write(f"Cost {result['cost']:.0f}\n")
    print(f"Solution enregistrée : {out_path}")
    return out_path

print("Fonction save_solution_tw définie !")

Fonction save_solution_tw définie !


## Tests rapides sur quelques instances Solomon (ex: C101, R101, RC101)

Sélectionnez les fichiers .txt Solomon dans `data/` et testez le solveur.

In [14]:
# Test rapide sur C101.txt
test_instance = 'data/cvrplib/Vrp-Set-Solomon/C101.txt'

print(f"Test sur l'instance: {os.path.basename(test_instance)}")

# Résoudre l'instance
results = []
try:
    res = solve_cvrptw(test_instance)
    save_solution_tw(res)
    results.append(res)
    print(f"✓ {os.path.basename(test_instance)} terminé")
except Exception as e:
    print(f"✗ Erreur sur {test_instance}: {e}")
    import traceback
    traceback.print_exc()

print(f"\n\nComplété la résolution de {len(results)} instance(s) !")


Test sur l'instance: C101.txt

Solving: C101.txt
Dimension: 101 nodes; Capacity: 200
Initial cost: 2359.80; Routes: 24
Après VND: 968.01
Final cost: 859.82
Final routes: 24; Time: 238.26 sec
Optimal: 827.3; Gap: 3.93%
Solution enregistrée : solutions_TW\C101_computed.sol
✓ C101.txt terminé


Complété la résolution de 1 instance(s) !


## Résumé et statistiques

Calcul d'un résumé simple des résultats pour un ensemble d'instances testées.

## Statistiques de résumé

In [ ]:
# Créer le rapport de résumé
import pandas as pd

summary_data = []
for result in results:
    summary_data.append({
        'Instance': result['instance_name'],
        'Coût Calculé': f"{result['cost']:.2f}",
        'Coût Optimal': f"{result['optimal_cost']:.2f}" if result['optimal_cost'] else 'N/A',
        'Écart (%)': f"{result['gap_pct']:.2f}" if result['gap_pct'] is not None else 'N/A',
        'Routes': result['n_routes'],
        'Temps (s)': f"{result['time_seconds']:.2f}"
    })

df_summary = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("RÉSUMÉ DES RÉSULTATS")
print("="*80)
print(df_summary.to_string(index=False))

# Calculer les statistiques pour les instances avec optimal connu
gaps = [r['gap_pct'] for r in results if r['gap_pct'] is not None]
if gaps:
    print(f"\n{'='*80}")
    print("STATISTIQUES D'ÉCART PAR RAPPORT À L'OPTIMAL")
    print("="*80)
    print(f"  Écart moyen     : {np.mean(gaps):.2f}%")
    print(f"  Écart médian    : {np.median(gaps):.2f}%")
    print(f"  Écart minimum   : {np.min(gaps):.2f}%")
    print(f"  Écart maximum   : {np.max(gaps):.2f}%")
    target_gap = config['quality']['target_gap_percentage']
    within_target = sum(g <= target_gap for g in gaps)
    print(f"  Instances ≤ {target_gap}%  : {within_target}/{len(gaps)} ({100*within_target/len(gaps):.1f}%)")
    print("="*80)

# Sauvegarder le résumé en CSV
summary_path = 'solutions_TW_ALNS_TABU/summary_results.csv'
df_summary.to_csv(summary_path, index=False)
print(f"\n✓ Résumé sauvegardé dans: {summary_path}")



RÉSUMÉ DES RÉSULTATS
Instance Coût Calculé Coût Optimal Écart (%)  Routes Temps (s)
C101.txt       859.82       827.30      3.93      24    238.26

STATISTIQUES D'ÉCART PAR RAPPORT À L'OPTIMAL
  Écart moyen     : 3.93%
  Écart médian    : 3.93%
  Écart minimum   : 3.93%
  Écart maximum   : 3.93%
  Instances ≤ 7.0%  : 1/1 (100.0%)

✓ Résumé sauvegardé dans: solutions_TW/summary_results.csv
